TEST BOUNDING BOX

In [1]:
import numpy as np
import cv2  
from mss import mss

bounding_box = {'top': 450, 'left': 900, 'width': 120, 'height': 200}

sct = mss()

while True:
    sct_img = sct.grab(bounding_box)
    cv2.imshow('screen', np.array(sct_img))

    key = cv2.waitKey(1)
    if key==ord('q'):
        break
        
cv2.destroyAllWindows()

AUTO FISHING

lower_red - [185, 10, 10]
upper_red - [225, 50, 50]

underwater_lower_red - [70, 50, 100]
underwater_upper_red - [100, 75, 125]

In [3]:
import numpy as np
import cv2  
from mss import mss
import time
import mouse

sct = mss()

lower_red = np.array([185, 10, 10])
upper_red = np.array([225, 50, 50])

underwater_lower = np.array([70, 50, 100])
underwater_upper = np.array([100, 75, 125])

line_cast = False

bite_start = 0
bite_end = 0
bite_min_time = .5
elapsed_bite_time = 0
bite = False

recast_start = 0
recast_end = 0
recast_time = .75
elapsed_recast_time = 0
recast = False

while True:
    sct_img = sct.grab(bounding_box)
    img_rgb = cv2.cvtColor(np.array(sct_img), cv2.COLOR_BGR2RGB)
    above_mask = cv2.inRange(img_rgb, lower_red, upper_red)
    under_mask = cv2.inRange(img_rgb, underwater_lower, underwater_upper)
    
    # detect if color is in capture
    redCount = cv2.findContours(above_mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    underwaterRedCount = cv2.findContours(under_mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    
    if len(redCount) > 0:
        if line_cast == False:
            line_cast = True
            print("Line Cast")
    else:
        line_cast = False
        
        
    # Hook is underwater     
    if len(underwaterRedCount) > 0:
        if bite == False:
            bite = True
            bite_start = time.time()
        else:
            bite_end = time.time()
            elapsed_bite_time = bite_end - bite_start
            
            if elapsed_bite_time > bite_min_time:
                print("BITE")
                mouse.click('right')
                bite = False
                recast = True                
    else:
        bite = False
        
    if recast:
        if recast_start == 0:
            recast_start = time.time()
        else:
            recast_end = time.time()
            elapsed_recast_time = recast_end - recast_start
            
            if elapsed_recast_time > recast_time:
                mouse.click('right')
                recast = False
                recast_start = 0
    
    cv2.imshow('screen', np.array(sct_img))
    cv2.imshow('Line Cast', above_mask)
    cv2.imshow('Bite', under_mask)

    key = cv2.waitKey(1)
    if key==ord('q'):
        break
        
cv2.destroyAllWindows()